# Yival supports multi-model now 🎉🎉
We can quickly experience the powerful features of Yival by visualizing some animal stories using Midjourney.

![hello](https://uninaruto.oss-cn-shanghai.aliyuncs.com/img/816211693805398_.pic_hd.jpg)

# Configure your OpenAI API key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''

# Configure your [the next leg key](https://www.thenextleg.io/)

We provide this key for user to try our product🐯

# Configure your OpenAI API key

In [ ]:
os.environ['MIDJOURNEY_TOKEN'] = '7ab0c4ac-4be4-419d-8656-ddb251383e17'

# MidJourney demo

```
custom_function: demo.animal_story.image_generation
description: Generated experiment config
dataset:
  data_generators:
    openai_prompt_data_generator:
      chunk_size: 100000
      diversify: true
      prompt:
          "Please provide a concrete and realistic test case as a dictionary for function invocation using the ** operator.
          Only include parameters, excluding description and name.
          Ensure it's succinct and well-structured.
          **Only provide the dictionary.**"
      input_function:
        description:
          Given the species of an animal and its character, generate a corresponding story
        name: animal_story_generation
        parameters:
          species: str
          character: str
          drawing_style: str
      number_of_examples: 3
      model_name: gpt-4
      output_path: animal_story.pkl
  source_type: machine_generated

variations:
  - name: task
    generator_name: openai_prompt_based_variation_generator
    generator_config:
      model_name: gpt-4
      number_of_variations: 3
      diversify: true
      max_tokens: 2000
      variables: null
      prompt: 
        - content: |-
            Your object is to construct a concise instruction prompt for GPT-4. This prompt will instruct GPT-4 as a gentle, imaginative children's writer to write all kinds of cute, kid-friendly stories based on animal species and personalities for an audience of YOUNG ADULTS and TEENAGERS.
            Points to emphasize in your instruction:
              -  GPT4 Answers should have a short storyline and should be sufficiently imaginative, answers should focus on the location in which the story takes place with its style of drawing, the location and style of drawing can be contrary to the laws of reality, dream-like or realistic.
              -   The story must be kept lively and as short, short, short, short, short as possible. The story should have a variety of backgrounds + drawing styles and fully emphasize the location where the story takes place; it can be dreamlike and generally bizarre.
              -  Be sure to use words that are as easy to understand as possible so TEENAGERS can easily understand the story.
              -   Please limit the output to within 300 words
            - {animal_species} represent the species of the animal
            - {animal_character} represent the character of the animal
            - At all times, gpt-4 must give a response as an animal story

            Craft your instruction Ensure that GPT-4 is able to fully understand the types and personalities of small animals and write engaging and fully pictorial stories.
            keep your output crisp: only the prompt, devoid of any extraneous content.

          role: system

evaluators:
  - evaluator_type: individual
    metric_calculators:
      - method: AVERAGE
    name: openai_prompt_based_evaluator
    display_name: story_quality
    prompt: |-
      You are assessing a submitted story based on a given task based on a criterion. Here is the data:
      - Task: Given the species of an animal and corresponding character, generate one story.
      - Does the story demonstrate creativity, coherence, and engaging narrative? A well-written story should captivate the reader and effectively convey a compelling narrative with clear structure and character development.
      [Input]: animal_species : {species}, animal_character: {character}
      [Result]: {raw_output}
      Answer the question by selecting one of the following options:
      A The story is of very low quality and lacks coherence and engagement.
      B The story has some issues with coherence and engagement, but has potential for improvement.
      C The story is of satisfactory quality, but there is room for improvement.
      D The story is well-written and demonstrates good coherence and engagement.
      E The story is exceptionally well-written, demonstrating strong coherence and engagement.
    choices: ["A", "B", "C", "D", "E"]
    # model_name specify the llm model , e.g. a16z-infra/llama-2-13b-chat:9dff94b1bed5af738655d4a7cbcdcde2bd503aa85c94334fe1f42af7f3dd5ee3
    model_name: gpt-4
    description: "evaluate the quality of the generated story"
    scale_description: "0-4"
    choice_scores:
      A: 0
      B: 1
      C: 2
      D: 3
      E: 4

human_rating_configs:
  - name: relevance
    instructions: Rate the relevance of the image to the story.
    scale: [1, 5]

  - name: image_quality
    instructions: Rate the quality of the generated image.
    scale: [1, 5]


selection_strategy:
  ahp_selection:
    criteria:
      - "openai_prompt_based_evaluator: story_quality"
      - average_token_usage
      - average_latency
    criteria_maximization:
      "openai_prompt_based_evaluator: story_quality": true
      average_latency: false
      average_token_usage: false
    criteria_weights:
      "openai_prompt_based_evaluator: story_quality": 1
      average_latency: 0.0
      average_token_usage: 0.0
    # normalize_func: "z-score"

improver:
  name: "optimize_by_prompt_improver"
  model_name: "gpt-4"
  max_iterations: 2
  improve_var: ["task"]
  head_meta_instruction: |
    Now you will help me generate a prompt which is used to generate a corresponding
    story according to the species of an animal which is [animal_species] and its character [animal_character]. 
    I already have some prompt and its evaluation results :
    
  end_meta_instruction: |
    Give me a new prompt that is different from all pairs above, and has a evaluation value higher than any of above.
```

In this demo , we mainly focus on writing stories accoding to animal species and character , and then midjourney will generate the corresponding cover of this story.

For dataset , opanai_prompt_data_generator will generate the animal species and its character like { cat , brave }

For variation , we write one prompt that generate generate-animal-story-template.

We format this template with { species , character } and generate detailed stories through GPT-4 , and with Midjourney we visualzie this story.

Also , we use opro_improver to automatically improve our prompt, which you can find more information in yival posts.

![image](https://uninaruto.oss-cn-shanghai.aliyuncs.com/img/818501693828594_.pic.jpg)

# Multi-model YIVAL !

In [ ]:
!poetry run yival run /content/YiVal/demo/configs/animal_story.yml